*please execute the cell below before starting the tutorial by selecting it and pressing Ctrl+Enter*

In [ ]:
# imports correct environment
from aiida import load_profile
load_profile()

# imports load_node() 
from aiida.orm import load_node

# Modifying a Fleur input file

To modify an existing `FleurinpData`, we need to load it first. Please, do in via `load_node()` function and proceed to the next section:
<!-- fleurinp = load_node(xxx) -->

In [ ]:
# you need to modify this - insert a PK of any FleurinpData
fleurinp = load_node(FLEURINP_PK)

## FleurinpModifier

The stored `FleurinpData` can not be modified in-place because it was sealed when it was stored in the database. Therefore we always need to create a new `FleurinpData` object to change an existing one. 

To make changes and store the result in the database, AiiDA-Fleur contains a `FluerinpModifier` class.

To start a process of `FleurinpData` modification, we need to import the `FleurinpModifier` class first and initialise an instance:

In [ ]:
# we can also import it using DataFactory:
# from aiida.plugins import DataFactory
# FleurinpModifier = DataFactory('fleur.fleurinpmodifier')

from aiida_fleur.data.fleurinpmodifier import FleurinpModifier

fleurmode = FleurinpModifier(fleurinp)

Now we need to work with methods belonging to the class to perform a modification. One needs to do two steps:

1. Register all required changes
2. Apply them and store the result in the database

### Registration methods

#### set_inpchanges()

Probably the simplest way to register a change is to use `set_inpchanges()` that replaces known attributes. For this, one needs to pass a <span style="color:red">key</span>: <span style="color:green">value</span> dictionary into the method call. A key usually corresponds to the name of an attribute in the `inp.xml` file. All supported attribute names can be found in the [documentation](https://aiida-fleur.readthedocs.io/en/latest/module_guide/tools.html#aiida_fleur.tools.xml_util.set_inpchanges).

To begin with, we want to set `itmax` to `30` and `minDistance` to `0.00002`:

In [ ]:
fleurmode.set_inpchanges({'itmax': 30, 'minDistance' : 0.00002})

One can also provide a python dictionary with the parameter names and their values you would like to change:

In [ ]:
change_dict = {
    'dos'       : True, 
    'ndir'      : -1, 
    'minEnergy' : -0.8,
    'maxEnergy' : 0.8, #### set_inpchanges()
    'sigma'     : 0.005,
}

fleurmode.set_inpchanges(change_dict)

#### changes preview

Note, the changes are in stock and **not applied yet**. You can make a preview of the resulting `inp.xml` by:

In [ ]:
fleurmode.show(validate=True)   #display=False

`validate=True` means the resulting `inp.xml` will be validated against the schema: if you specify changes leading to corrupted `inp.xml` file you will be informed. See an example below:

In [ ]:
# If the changes are not valid we will get an error
fleurmode_fail = FleurinpModifier(fleurinp)
fleurmode_fail.set_inpchanges({'itmax': -10, 'minDistance' : 0.001})
fleurmode_fail.show(validate=True, display=False)

In [ ]:
# or if you misstype a key wrong
fleurmode_fail.undo()
fleurmode_fail.set_inpchanges({'itma': 10, 'minDistance' : 0.001})
fleurmode_fail.show(validate=True, display=False)

#### Methods for species manipulation

Change muffin tin radii, or any species parameters you have to parse a nested dict with the subtags

In [ ]:
# you need to modify this - replace 'W-1' to a correct specie name 
fleurmode.set_species('W-1', {'mtSphere' : {'radius' : 3.5, 'gridPoints' : 841}, 
                              'atomicCutoffs' : {'lmax' : 9, 'lnonsphr' : 6}})


See the result

In [ ]:
fleurmode.show(validate=True)

#### Registered changes management

To print out the list of registered changes, run:

In [ ]:
fleurmode.changes()

all these changes are not applied and we can revert them:

In [ ]:
fleurmode.undo()   # drops last registered change

In [ ]:
fleurmode.changes()

In [ ]:
fleurmode.undo()

In [ ]:
fleurmode.changes()

### Apply changes and store in the database

With `freeze()` method we store a new `FleurinpData` object in the database applying all registered changes. `freeze()` return a stored `FleurinpData` object:

In [ ]:
fleurinp_modified = fleurmode.freeze()
print('The PK of the stored FleurinpData is {}'.format(fleurinp_modified.pk))

## XML registration methods

`FleurinpModifier` contains more general xml methods to deal with tags, attributes and text of an xml file.
They require some knoledge on the internal structure of the `inp.xml` file, however, provide more general and flexible tools for inp.xml manipulation.

In this tutorial we will cover only a few existing methods, for all of them see [the documentation](https://aiida-fleur.readthedocs.io/en/latest/user_guide/plugin/fleurinp_data.html#id1).

The first example is changing `itmax` in the `inp.xml` file. We did it already via `set_inpchanges()` methods above, but there is another way to do it:

In [ ]:
xpathn = '/fleurInput/calculationSetup/scfLoop'
fleurmode.xml_set_all_attribv(xpathn, 'itmax', 29)
fleurmode.show(validate=True) # preview the result

Another example is changing the total number of k-points by replacing `kPointCount` tag entirely. It can be done using `replace_tag()`:

In [ ]:
from lxml import etree

kpoint_xpath = '/fleurInput/calculationSetup/bzIntegration/kPointList'

nkpts = 400
gamma='F'

new_kpt_tag = etree.Element('kPointCount', count="{}".format(nkpts), gamma="{}".format(gamma))
fleurmode.replace_tag(kpoint_xpath, new_kpt_tag)
fleurmode.show(validate=True) # preview the result
fleurmode.undo() # drop the last change

or more simply be a pre-defined method `set_nkpts()`:

In [ ]:
nkpts = 800
fleurmode.set_nkpts(count=nkpts)
fleurmode.show(validate=True) # preview the result

In conclusion, there are two types of registration methods: pre-defined and xml ones. Pre-defined changes already know where to find a certain attribute that a user wants to change. In contrast, XML methods can be more flexible because they require an xml path to work.

In next tutorial we are going to learn how to generate `inp.xml` and corresponding `FleurinpData` object using the `inpgen` code.